In [2]:
from typing import Tuple
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pandas as pd

import numpy as np

import torch

from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Optimizer, Adam
from torch.optim.lr_scheduler import LRScheduler
import wandb

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
torch.set_default_dtype(torch.float64)

In [5]:
# df = pd.concat([pd.read_csv("more-elements/more-elements.csv"), pd.read_csv("new-more-data/new-more-data.csv")])
# df
df = pd.read_pickle("half_data.pkl")
df

,0,1,2,3,4,5,6,7,8,9,...,1000,element_1,element_2,element_3,element_1_ratio,element_2_ratio,element_3_ratio,temp,pressure,air_ratio
0,1.402026e-21,1.650584e-21,1.910661e-21,2.264304e-21,2.829955e-21,3.673546e-21,4.745899e-21,5.773241e-21,6.446840e-21,6.741727e-21,...,3.466256e-20,17,23,-1,0.440157,0.559843,0.000000,283.0,0.775,0.0
1,4.102767e-22,4.371429e-22,4.698476e-22,5.105580e-22,5.626400e-22,6.315813e-22,7.266133e-22,8.610945e-22,1.048941e-21,1.323529e-21,...,2.481710e-21,0,19,-1,0.428047,0.571953,0.000000,303.0,0.800,0.6
2,3.610634e-23,2.739156e-23,1.628379e-23,1.195799e-23,1.069667e-23,1.058083e-23,1.121691e-23,1.274356e-23,1.515703e-23,1.882304e-23,...,2.756785e-21,2,11,13,0.342547,0.376666,0.280788,283.0,0.500,0.6
3,1.016487e-22,1.356304e-22,1.894228e-22,3.406116e-22,8.410447e-22,2.039525e-21,1.635235e-21,6.587985e-22,3.087047e-22,2.124933e-22,...,1.240249e-22,6,10,13,0.053299,0.321178,0.625524,293.0,0.900,0.6
4,1.637227e-21,2.113385e-21,2.633061e-21,3.098701e-21,3.509187e-21,3.569827e-21,3.557713e-21,3.356628e-21,3.389955e-21,3.744591e-21,...,3.614986e-23,9,18,24,0.470739,0.251465,0.277796,273.0,0.500,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183548,1.720379e-22,2.933964e-22,4.190668e-22,5.128756e-22,6.537258e-22,6.598291e-22,7.231058e-22,6.818061e-22,8.002978e-22,1.130003e-21,...,8.585612e-23,9,17,23,0.401527,0.454117,0.144356,313.0,0.500,0.6
183549,2.392508e-21,1.998411e-21,1.320957e-21,9.131813e-22,6.948151e-22,5.720024e-22,4.987314e-22,4.533583e-22,4.270046e-22,4.143369e-22,...,2.460996e-21,4,8,16,0.348675,0.389984,0.261341,323.0,1.000,0.6
183550,1.947286e-22,2.092204e-22,2.261748e-22,2.462494e-22,2.703056e-22,2.995569e-22,3.357735e-22,3.814871e-22,4.405433e-22,5.189812e-22,...,1.838447e-23,1,7,14,0.116401,0.430989,0.452610,313.0,0.100,0.6
183551,4.523992e-22,6.372981e-22,1.661855e-21,1.874233e-22,1.226359e-22,1.804612e-22,3.107570e-22,1.559673e-21,3.250324e-21,2.427107e-21,...,1.508687e-21,15,18,23,0.422131,0.441890,0.135979,283.0,0.200,0.3


In [6]:
# elems = df["element_1"].unique()
# elem2id = {}
# for i, elem in enumerate(elems):
#     elem2id[elem] = i

# df.fillna(
#     {
#         "element_1": "UNKNOWN",
#         "element_2": "UNKNOWN",
#         "element_3": "UNKNOWN",
#         "element_1_ratio": 0.0,
#         "element_2_ratio": 0.0,
#         "element_3_ratio": 0.0,
#     },
#     inplace=True,
# )

# df.dropna(axis=0, inplace=True)

# elem2id

In [7]:
# df["element_1"] = df["element_1"].apply(
#     lambda x: elem2id[x] if x != "UNKNOWN" else -1
# )
# df["element_2"] = df["element_2"].apply(lambda x: elem2id[x] if x != "UNKNOWN" else -1)
# df["element_3"] = df["element_3"].apply(lambda x: elem2id[x] if x != "UNKNOWN" else -1)
# df

In [8]:
print(len(df))
df = df[(df[[str(i) for i in range(1001)]] > 0).all(axis=1)]
print(len(df))

183553
183553


In [9]:
len(df[(df["element_1"] == 0) | (df["element_2"] == 0) | (df["element_3"] == 0)]) / len(df)

0.11451733286843582

In [10]:
class CustomSpectraDataset(Dataset):
    def __init__(self, data: pd.DataFrame, device="cuda:0") -> None:
        self.data = data
        self.elements = self.data["element_1"].unique()
        self.air_ratios = self.data.air_ratio.to_numpy(dtype=np.float64)

        self.spectras = torch.log(
            torch.tensor(df[[str(i) for i in range(1001)]].to_numpy(dtype=np.float64))
        ).to(device)

        self.ratios = torch.tensor(
            df[["element_1_ratio", "element_2_ratio", "element_3_ratio"]].to_numpy(
                dtype=np.float64
            )
        ).to(device)

        self.element_indices = df[["element_1", "element_2", "element_3"]].to_numpy(
            dtype=np.float64
        )

        self.elements_distributions = torch.zeros(
            [len(self.data), len(self.elements) + 1], dtype=torch.float64
        ).to(device)

        for idx in range(len(self.data)):
            indices = self.element_indices[idx, :]
            indices = indices[indices != -1]
            self.elements_distributions[idx, indices] = self.ratios[idx][
                range(indices.shape[0])
            ] * (1 - self.air_ratios[idx])
            self.elements_distributions[idx, -1] = self.air_ratios[idx]

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        spectra = self.spectras[idx]
        elements_distribution = self.elements_distributions[idx]

        return spectra, elements_distribution

In [11]:
class THzResNetBlock(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: int = 3,
        stride: int = 1,
        activation: nn.Module = nn.ReLU(),
        dropout: float = 0.05,
    ) -> None:
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        assert kernel_size % 2 == 1
        self.layers = nn.Sequential(
            nn.Conv1d(
                in_channels,
                out_channels,
                kernel_size=kernel_size,
                padding=kernel_size // 2,
                stride=stride,
            ),
            nn.Dropout(p=dropout),
            activation,
            nn.Conv1d(
                out_channels,
                out_channels,
                kernel_size=kernel_size,
                padding=kernel_size // 2,
                stride=stride,
            ),
            nn.Dropout(p=dropout),
            activation,
        )
        self.skip_connection = nn.Conv1d(
            in_channels=in_channels, out_channels=out_channels, kernel_size=1
        )
        self.post_processing = nn.Sequential(
            nn.BatchNorm1d(self.out_channels),
            activation,
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        unprocessed_result = self.layers(x) + self.skip_connection(x)
        return self.post_processing(unprocessed_result)

In [12]:
class THzBottleNeck(nn.Module):

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: int = 3,
        stride: int = 1,
        activation: nn.Module = nn.ReLU(),
        dropout: float = 0.05,
    ) -> None:
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        assert kernel_size % 2 == 1
        self.layers = nn.Sequential(
            nn.Conv1d(
                in_channels,
                in_channels // 4,
                kernel_size=1,
                padding=0,
                stride=1,
            ),
            nn.Dropout(p=dropout),
            activation,
            nn.Conv1d(
                in_channels // 4,
                in_channels // 4,
                kernel_size=kernel_size,
                padding=kernel_size // 2,
                stride=stride,
            ),
            nn.Dropout(p=dropout),
            activation,
            nn.Conv1d(
                in_channels // 4,
                out_channels,
                kernel_size=1,
                padding=0,
                stride=1,
            ),
            activation,
            nn.Dropout(p=dropout),
        )
        self.skip_connection = nn.Conv1d(
            in_channels=in_channels, out_channels=out_channels, kernel_size=1
        ) if in_channels != out_channels else lambda x: x
        self.post_processing = nn.Sequential(
            nn.BatchNorm1d(self.out_channels),
            activation,
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        unprocessed_result = self.layers(x) + self.skip_connection(x)
        return self.post_processing(unprocessed_result)

In [40]:
class THzCNN(nn.Module):
    def __init__(self, n_elements: int) -> None:
        super().__init__()
        self.n_elements = n_elements
        self.net = nn.Sequential(
            # input_shape: (batch_size, 1, 1001)
            nn.BatchNorm1d(1),
            THzResNetBlock(in_channels=1, out_channels=64, kernel_size=7),
            THzBottleNeck(in_channels=64, out_channels=64, kernel_size=3),
            THzBottleNeck(in_channels=64, out_channels=64, kernel_size=3),
            
            nn.MaxPool1d(kernel_size=4),
            # input_shape: (batch_size, 64, 250)
            THzResNetBlock(in_channels=64, out_channels=128, kernel_size=3),
            THzBottleNeck(in_channels=128, out_channels=128, kernel_size=3),
            THzBottleNeck(in_channels=128, out_channels=128, kernel_size=3),
            
            nn.MaxPool1d(kernel_size=5),
            # input_shape: (batch_size, 128, 50)
            THzResNetBlock(in_channels=128, out_channels=256, kernel_size=3),
            THzBottleNeck(in_channels=256, out_channels=256, kernel_size=3),
            THzBottleNeck(in_channels=256, out_channels=256, kernel_size=3),
            
            nn.MaxPool1d(kernel_size=5),
            # input_shape: (batch_size, 256, 10)
            THzResNetBlock(in_channels=256, out_channels=512, kernel_size=3),
            THzBottleNeck(in_channels=512, out_channels=512, kernel_size=3),
            THzBottleNeck(in_channels=512, out_channels=512, kernel_size=3),
            
            nn.MaxPool1d(kernel_size=5),
            # input_shape: (batch_size, 512, 2)
            nn.Flatten(),
            # linear head
            nn.Linear(512 * 2, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, self.n_elements),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)

In [30]:
import wandb.wandb_run


def train_epoch(
    model: nn.Module,
    dataloader: DataLoader,
    optimizer: Optimizer,
    loss_fn: nn.Module,
    scheduler: LRScheduler = None,
) -> float:
    model.train()
    total_loss = 0
    for spectra, target in tqdm(dataloader, desc="Training"):
        optimizer.zero_grad()
        pred = model(spectra[:, None, :])
        loss = loss_fn(pred, target)

        total_loss += loss.item()

        loss.backward()
        # nn.utils.clip_grad_norm_(model.parameters(), 10)
        optimizer.step()

        if scheduler is not None:
            scheduler.step()

    return total_loss / len(dataloader)


@torch.no_grad()
def val_epoch(
    model: nn.Module, dataloader: DataLoader, loss_fn: nn.Module
) -> float:
    model.eval()
    loss = 0
    mae = 0
    for spectra, target in tqdm(dataloader, desc="Validating"):

        pred = model(spectra[:, None, :])
        loss += float(loss_fn(pred, target).item())
        mae += float(nn.L1Loss()(nn.Softmax()(pred), target).item())

    return loss / len(dataloader), mae / len(dataloader)

In [31]:
def train(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    optimizer: Optimizer,
    n_epochs: int,
    loss_fn: nn.Module = nn.CrossEntropyLoss(reduction="mean"),
    scheduler: LRScheduler = None,
    run: wandb.wandb_run.Run = None
) -> None:

    for i in range(1, n_epochs + 1):
        train_loss = train_epoch(
            model, train_loader, optimizer, loss_fn, scheduler
        )

        val_loss, mae = val_epoch(model, val_loader, loss_fn)

        print(
            f"Epoch {i}: \n   Train loss: {train_loss:.5f}    |   Val loss: {val_loss:.5f}   |   Val MAE: {mae:.5f}\n"
        )
        if run:
            run.log({"train_loss": train_loss, "val_loss": val_loss, "MAE": mae})

In [16]:
train_df, test_df = train_test_split(df, test_size=0.15, random_state=42)

In [17]:
(train_df[[str(i) for i in range(1001)]] < 0).any()

0       False
1       False
2       False
3       False
4       False
        ...  
996     False
997     False
998     False
999     False
1000    False
Length: 1001, dtype: bool

In [18]:
lr = 1e-4
n_epochs = 30
batch_size = 32

In [19]:
train_dataset = CustomSpectraDataset(train_df)
val_dataset = CustomSpectraDataset(test_df)

In [20]:
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [21]:
torch.autograd.set_detect_anomaly(True)

In [32]:
run = wandb.init(
    # set the wandb project where this run will be logged
    project="course-work",
    save_code=True,
    group="CNN",
    name="Less dropout",
    notes="Model stopped overfitting. Use a little bit less dropout and log mae metric",
    config={
        "learning_rate": lr,
        "architecture": "CNN",
        "epochs": n_epochs,
        "batch_size": batch_size,
    },
)

In [23]:
from torch.optim.lr_scheduler import SequentialLR, LinearLR, CosineAnnealingWarmRestarts
from torch.optim import SGD

In [24]:
# lrs = []
# for i in range(50000):
#     scheduler.step()
#     lrs.append(optimizer.param_groups[0]["lr"])

In [25]:
# import matplotlib.pyplot as plt

# plt.plot(lrs)

In [41]:
net = THzCNN(val_dataset[0][1].shape[0])
net.to(device)

THzCNN(
  (net): Sequential(
    (0): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): THzResNetBlock(
      (layers): Sequential(
        (0): Conv1d(1, 64, kernel_size=(7,), stride=(1,), padding=(3,))
        (1): Dropout(p=0.05, inplace=False)
        (2): ReLU()
        (3): Conv1d(64, 64, kernel_size=(7,), stride=(1,), padding=(3,))
        (4): Dropout(p=0.05, inplace=False)
        (5): ReLU()
      )
      (skip_connection): Conv1d(1, 64, kernel_size=(1,), stride=(1,))
      (post_processing): Sequential(
        (0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU()
      )
    )
    (2): THzBottleNeck(
      (layers): Sequential(
        (0): Conv1d(64, 16, kernel_size=(1,), stride=(1,))
        (1): Dropout(p=0.05, inplace=False)
        (2): ReLU()
        (3): Conv1d(16, 16, kernel_size=(3,), stride=(1,), padding=(1,))
        (4): Dropout(p=0.05, inplace=False)
        (5): ReLU()
   

In [42]:
optimizer = Adam(net.parameters(), lr=lr)
# optimizer = SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=1e-4)
# scheduler1 = LinearLR(optimizer, start_factor=0.05, end_factor=1, total_iters=500)
# scheduler2 = CosineAnnealingWarmRestarts(optimizer, T_0=1500, T_mult=2)
# scheduler = SequentialLR(
#     optimizer, schedulers=[scheduler1, scheduler2], milestones=[500]
# )


In [43]:
print(device)

train(net, train_loader, val_loader, optimizer, n_epochs, scheduler=None, run=run)
# train(net, train_loader, val_loader, optimizer, n_epochs, scheduler=None, run=None)

cuda:0


Validating:   0%|          | 0/861 [00:00<?, ?it/s]c:\Users\whoee\Desktop\CourseWork\.venv\lib\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
Validating: 100%|██████████| 861/861 [00:57<00:00, 15.04it/s]


Epoch 1: 
   Train loss: 1.56225    |   Val loss: 1.41226   |   Val MAE: 0.02424



Validating: 100%|██████████| 861/861 [00:57<00:00, 15.04it/s]


Epoch 2: 
   Train loss: 1.38867    |   Val loss: 1.36462   |   Val MAE: 0.02303



...

Epoch 17: 
   Train loss: 1.24262    |   Val loss: 1.32256   |   Val MAE: 0.02190



Validating: 100%|██████████| 861/861 [00:57<00:00, 15.06it/s]


Epoch 18: 
   Train loss: 1.23442    |   Val loss: 1.32946   |   Val MAE: 0.02200



Training:  99%|█████████▉| 4826/4876 [25:17<00:15,  3.18it/s]


KeyboardInterrupt: 

In [ ]:
# from torch.profiler import profile, record_function, ProfilerActivity

In [ ]:
# with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
#     total_loss = 0
#     i = 0
#     for spectra, target in tqdm(train_loader, desc="Training"):
#         optimizer.zero_grad()
#         pred = net(spectra[:, None, :])
#         loss = nn.CrossEntropyLoss()(pred, target)

#         total_loss += loss.item()

#         loss.backward()
#         optimizer.step()
#         i += 1
#         if i >= 10:
#             break

    

In [ ]:
# print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

In [44]:
wandb.finish()

MAE,█▅▄▃▂▂▂▁▁▁▁▁▁▁▂▂▂▂
train_loss,█▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁
val_loss,█▅▄▃▂▂▂▁▁▁▁▁▁▁▂▂▂▃
MAE,0.022
train_loss,1.23442
val_loss,1.32946


In [45]:
torch.save(net.state_dict(), "./air-dropout.model")

In [46]:
# train on MSE
import wandb.wandb_run


def train_epoch(
    model: nn.Module,
    dataloader: DataLoader,
    optimizer: Optimizer,
    loss_fn: nn.Module,
    scheduler: LRScheduler = None,
) -> float:
    total_loss = 0
    for spectra, target in tqdm(dataloader, desc="Training"):
        optimizer.zero_grad()
        pred = model(spectra[:, None, :])
        loss = nn.MSELoss()(nn.Softmax()(pred), target)

        total_loss += loss_fn(pred, target).item()

        loss.backward()
        # nn.utils.clip_grad_norm_(model.parameters(), 10)
        optimizer.step()

        if scheduler is not None:
            scheduler.step()

    return total_loss / len(dataloader)


@torch.no_grad()
def val_epoch(model: nn.Module, dataloader: DataLoader, loss_fn: nn.Module) -> float:
    loss = 0
    mae = 0
    for spectra, target in tqdm(dataloader, desc="Validating"):

        pred = model(spectra[:, None, :])
        loss += float(loss_fn(pred, target).item())
        mae += float(nn.L1Loss()(nn.Softmax()(pred), target).item())

    return loss / len(dataloader), mae / len(dataloader)

In [47]:
run = wandb.init(
    # set the wandb project where this run will be logged
    project="course-work",
    save_code=True,
    group="CNN",
    name="MSE v2",
    notes="Train on MSE + log MAE metric",
    config={
        "learning_rate": lr,
        "architecture": "CNN",
        "epochs": n_epochs,
        "batch_size": batch_size,
    },
)

In [48]:
net = THzCNN(val_dataset[0][1].shape[0])
net.to(device)
optimizer = Adam(net.parameters(), lr=lr)

print(device)
train(net, train_loader, val_loader, optimizer, n_epochs, scheduler=None, run=run)

cuda:0


Validating: 100%|██████████| 861/861 [00:57<00:00, 15.04it/s]


Epoch 1: 
   Train loss: 1.79338    |   Val loss: 1.57833   |   Val MAE: 0.02863



Validating: 100%|██████████| 861/861 [00:59<00:00, 14.52it/s]


Epoch 2: 
   Train loss: 1.50944    |   Val loss: 1.46837   |   Val MAE: 0.02581



...

Epoch 11: 
   Train loss: 1.34573    |   Val loss: 1.36756   |   Val MAE: 0.02274



Validating: 100%|██████████| 861/861 [01:00<00:00, 14.16it/s]


Epoch 12: 
   Train loss: 1.34045    |   Val loss: 1.36761   |   Val MAE: 0.02289



Training:  47%|████▋     | 2304/4876 [12:47<14:16,  3.00it/s]


KeyboardInterrupt: 

In [49]:
wandb.finish()

MAE,█▅▃▃▂▂▁▁▁▁▁▁
train_loss,█▄▃▂▂▂▁▁▁▁▁▁
val_loss,█▄▃▂▂▂▁▁▁▁▁▁
MAE,0.02289
train_loss,1.34045
val_loss,1.36761


In [50]:
torch.save(net.state_dict(), "./air-dropout-mse.model")